In [43]:
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [44]:
(train_images, train_labels), (test_images, test_labels)=tf.keras.datasets.mnist.load_data()

In [45]:
train_images.resize(60000,784)
test_images.resize(10000,784)

In [46]:
train_labels_ohe=np.zeros((train_labels.size,train_labels.max()+1))
train_labels_ohe[np.arange(train_labels.size),train_labels]=1
test_labels_ohe=np.zeros((test_labels.size,test_labels.max()+1))
test_labels_ohe[np.arange(test_labels.size),test_labels]=1

In [47]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2 #// for integer division 

In [48]:
weights={
    "wc1":tf.Variable(tf.compat.v1.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2":tf.Variable(tf.compat.v1.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1":tf.Variable(tf.compat.v1.random_normal([input_size_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.compat.v1.random_normal([n_hidden,n_out]))
}
biases={
    "bc1":tf.Variable(tf.compat.v1.random_normal([n_conv1])),
    "bc2":tf.Variable(tf.compat.v1.random_normal([n_conv2])),
    "bh1":tf.Variable(tf.compat.v1.random_normal([n_hidden])),
    "bo":tf.Variable(tf.compat.v1.random_normal([n_out]))
}

In [49]:
def conv(x,weights,bias,strides=1):
    out=tf.nn.conv2d(x,weights,strides=[1,strides,strides,1],padding="SAME")
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [50]:
def cnn(x,weights,biases,keep_prob):

    x=tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
    conv1=conv(x,weights["wc1"],biases["bc1"],stride_conv1)
    conv1_pool=maxpooling(conv1,max_pool1_k)

    conv2=conv(conv1_pool,weights["wc2"],biases["bc2"],stride_conv2)
    conv2_pool=maxpooling(conv2,max_pool2_k)

    input_hidden=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
    hidden_output_before_activation=tf.add(tf.matmul(input_hidden,weights["wh1"]),biases["bh1"])
    hidden_output_before_dropout=tf.nn.relu(hidden_output_before_activation)

    hidden_output=tf.nn.dropout(hidden_output_before_dropout,keep_prob)
    output=tf.add(tf.matmul(hidden_output,weights["wo"]),biases["bo"])
    return output

In [51]:
x=tf.compat.v1.placeholder("float",[None,input_pixels])
y=tf.compat.v1.placeholder(tf.int32,[None,n_out])
keep_prob=tf.compat.v1.placeholder("float")
pred=cnn(x,weights,biases,keep_prob)

In [52]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

In [53]:
optimizer=tf.compat.v1.train.AdadeltaOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [54]:
sess=tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [55]:
batch_size=100
for k in range(25):
    n=int(train_images.shape[0]/batch_size)
    total_cost=0
    i=0
    for j in range(n):
        x_=train_images[i:i+batch_size][:]
        y_=train_labels_ohe[i:i+batch_size][:]
        i+=batch_size
        c,_=sess.run([cost,optimize],feed_dict={x:x_,y:y_,keep_prob:0.8})
        total_cost+=c
    print(total_cost)

17554182002.0
16140703222.0
14883729552.0
13803754534.0
12870981409.0
12111071721.0
11350972661.0
10710643861.0
10078930614.0
9471945080.0
8925703836.0
8434721400.0
7967338709.0
7531096457.0
7066952120.0
6734659559.0
6361280507.0
6021719654.0
5701994545.5
5445308559.5
5169971302.5
4911384278.0
4707435746.0
4493877029.5
4280966842.0


In [56]:
prediction=tf.argmax(pred,1)
correct_labels=tf.argmax(y,1)
correct_predictions=tf.equal(prediction,correct_labels)
prediction,correct_preds=sess.run([prediction,correct_predictions],feed_dict={x:test_images,y:test_labels_ohe,keep_prob:1})
correct_preds.sum()

980